In [1]:
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import pickle
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Flatten

import os

In [2]:
def calculate_and_log_training_time(modelname, start_time, end_time):
    training_duration = end_time - start_time
    hours, rem = divmod(training_duration, 3600)
    minutes, seconds = divmod(rem, 60)
    
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    filepath = os.path.join(model_dir, "trainingtime.txt")
    with open(filepath, "w") as f:
        f.write(f"Training took {int(hours):02d}:{int(minutes):02d}:{seconds:02f} (hh:mm:ss).")

def save_model_config_with_optimizer(model, modelname):
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    filepath = os.path.join(model_dir, "model_config.txt")
    with open(filepath, 'w') as f:
        for layer in model.layers:
            f.write(f"Layer: {layer.name}\n")
            f.write(f"Config: {layer.get_config()}\n\n")
        
        optimizer_config = model.optimizer.get_config()
        f.write("Optimizer Config:\n")
        f.write(str(optimizer_config))

def save_model(model, modelname):
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    model_path = os.path.join(model_dir, modelname + ".keras")
    model.save(model_path)

def save_performance_metrics(history, modelname):
    model_dir = modelname
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    filepath = os.path.join(model_dir, 'performance_metrics.csv')
    pd.DataFrame(history.history).to_csv(filepath)

def plot_loss_and_metric(history, metric_name='accuracy', model_name='model'):
    model_dir = model_name
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    # First plot: Training & validation loss
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(history.history['loss'], label='Train Loss')
    ax.plot(history.history['val_loss'], label='Validation Loss')
    ax.set_title('Model Loss')
    ax.set_ylabel('Loss')
    ax.set_xlabel('Epoch')
    ax.legend(loc='upper right')
    ax.grid(True)
    loss_plot_filename = os.path.join(model_dir, f'{model_name}_loss_plot.jpg')
    fig.savefig(loss_plot_filename)
    print(f"Loss plot saved as: {loss_plot_filename}")
    plt.close(fig)
    
    # Second plot: Training & validation metric
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(history.history[metric_name], label=f'Train {metric_name.capitalize()}')
    ax.plot(history.history[f'val_{metric_name}'], label=f'Validation {metric_name.capitalize()}')
    ax.set_title(f'Model {metric_name.capitalize()}')
    ax.set_ylabel(metric_name.capitalize())
    ax.set_xlabel('Epoch')
    ax.legend(loc='upper left')
    ax.grid(True)
    metric_plot_filename = os.path.join(model_dir, f'{model_name}_{metric_name}_plot.jpg')
    fig.savefig(metric_plot_filename)
    print(f"Metric plot saved as: {metric_plot_filename}")
    plt.close(fig)

def save_model_and_config_and_metrics(model, history, modelname = "model"):
    save_model_config_with_optimizer(model, modelname = modelname)
    #save_model(model, modelname = modelname)
    save_performance_metrics(history, modelname = modelname)
    plot_loss_and_metric(history, metric_name='mae', model_name= modelname)

## Greyscale

In [3]:
path = r"C:\Users\busjo\Documents\JADS\Semester 2\Deep Learning\Project\Part1_Processed_Grey.pkl"
# Open the pickle file in binary mode
with open(path, 'rb') as file:
    # Load the content of the file into a variable
    grey_data = pickle.load(file)
    
sampled_grey_data = grey_data.sample(n=1500)

X = np.stack(sampled_grey_data['Image'].values)  # Converts a column of arrays into a single numpy array
X = X = X.reshape(X.shape[0], -1)
y = sampled_grey_data['Age'].values
#X = list(sampled_grey_data['Image'].values)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
model = Sequential([Dense(2048, activation = 'relu', input_shape=(65536,)),
Dense(512, activation = 'relu' ),
Dense(128, activation = 'relu' ),
Dense(32, activation = 'relu' ),
Dense(1)])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

start_time = time.time()
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
end_time = time.time()
modelname = "ann_grey"
calculate_and_log_training_time(modelname = modelname, start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = modelname)

c:\Users\busjo\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - loss: 9217.4170 - mae: 64.7921 - val_loss: 1759.6501 - val_mae: 35.3859
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - loss: 1097.3640 - mae: 26.9959 - val_loss: 1565.7611 - val_mae: 33.4004
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - loss: 1243.8094 - mae: 28.4231 - val_loss: 922.5847 - val_mae: 26.2375
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 993ms/step - loss: 741.1948 - mae: 22.2873 - val_loss: 743.2515 - val_mae: 23.2870
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 992ms/step - loss: 862.8376 - mae: 23.5564 - val_loss: 757.9734 - val_mae: 20.9726
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - loss: 729.1170 - mae: 21.7215 - val_loss: 668.3164 - val_mae: 20.9323
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - loss: 690.5264 - mae: 20.9721 - val_loss: 1132.1041 - val_mae: 28.8099
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - loss: 863.8949 - mae: 23.3934 - val_loss: 652.4696 - val_mae: 21.0792
Epoch 9/10
3

## RGB

In [5]:
path = r"C:\Users\busjo\Documents\JADS\Semester 2\Deep Learning\Project\Part1_Processed_RGB.pkl"
# Open the pickle file in binary mode
with open(path, 'rb') as file:
    # Load the content of the file into a variable
    RGB_data = pickle.load(file)
    
sampled_RGB_data = RGB_data.sample(n=1000)

X = np.stack(sampled_RGB_data['Image'].values)  # Converts a column of arrays into a single numpy array
X = X = X.reshape(X.shape[0], -1)
y = sampled_RGB_data['Age'].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
model2 = Sequential([Dense(2048, activation = 'relu', input_shape=(196608,)),
Dense(512, activation = 'relu' ),
Dense(128, activation = 'relu' ),
Dense(32, activation = 'relu' ),
Dense(1)])


model2.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

start_time = time.time()
history2 = model2.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)
end_time = time.time()

modelname = "ann_rgb"
calculate_and_log_training_time(modelname = modelname, start_time = start_time, end_time = end_time)
save_model_and_config_and_metrics(model = model, history = history,  modelname = modelname)

c:\Users\busjo\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 165s 5s/step - loss: 76285.6719 - mean_absolute_error: 130.6575 - val_loss: 952.3283 - val_mean_absolute_error: 22.0873
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - loss: 971.5742 - mean_absolute_error: 24.6843 - val_loss: 745.7155 - val_mean_absolute_error: 23.9387
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - loss: 1588.2229 - mean_absolute_error: 31.7824 - val_loss: 1008.0150 - val_mean_absolute_error: 23.2296
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - loss: 656.2308 - mean_absolute_error: 20.6905 - val_loss: 797.0519 - val_mean_absolute_error: 24.6242
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - loss: 872.7722 - mean_absolute_error: 23.0500 - val_loss: 2408.7200 - val_mean_absolute_error: 43.2442
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - loss: 1319.5305 - mean_absolute_error: 29.1235 - val_loss: 552.5101 - val_mean_absolute_error: 19.9452
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - loss: 639.1434 -